# Birdsong Classification #

This project is an attempt to determine the species of a bird given a short recording of it's birdsong. The data was obtained here: https://www.kaggle.com/datasets/vinayshanbhag/bird-song-data-set


The data contains over 5000 3-second recordings of birdsongs in wav format, and each of which belongs to one of five species:

* Bewick's Wren (Thyromanes Bewickii)
* Northern Cardinal (Cardinalis Cardinalis)
* American Robin (Turdus Migratorius)
* Song Sparrow (Melospiza Melodia)
* Northern Mockingbird (Mimus Polyglottos) 

In [80]:
import os

if not os.path.exists('./features'):
    os.mkdir('./features')
    
if not os.path.exists('./figures'):
    os.mkdir('./figures')

if not os.path.exists('./parameters'):
    os.mkdir('./parameters')

In [1]:
import pandas as pd
data = pd.read_csv('./archive/bird_songs_metadata.csv')[['id','genus','species','filename']]
print(data)

          id       genus     species      filename
0     557838  Thryomanes    bewickii  557838-0.wav
1     557838  Thryomanes    bewickii  557838-1.wav
2     557838  Thryomanes    bewickii  557838-4.wav
3     557838  Thryomanes    bewickii  557838-5.wav
4     557838  Thryomanes    bewickii  557838-6.wav
...      ...         ...         ...           ...
5417   11713  Cardinalis  cardinalis   11713-8.wav
5418   11713  Cardinalis  cardinalis  11713-10.wav
5419   11713  Cardinalis  cardinalis  11713-14.wav
5420   11713  Cardinalis  cardinalis  11713-15.wav
5421   11713  Cardinalis  cardinalis  11713-16.wav

[5422 rows x 4 columns]


## Feature Engineering 


### 1) Mel-Frequency Cepstrum Coefficients
Many feature engineernig methods for audio files exist. One of the most common is the **Mel-Frequency Cepstrum Coefficients (MFCC)**. We will begin with this, but also experiment with others.

There is significant freedom in specifying the window size and window shift length in the spectrogram. It will be important to balance information loss with high dimensionality in this feature space.

In [17]:
import torchaudio
import numpy as np

sample_rate = 22050 # sample rate given from dataset description
win_length_seconds = 0.04 # specify 40ms window sizes

win_length = int(sample_rate*win_length_seconds) 
hop_length = int(sample_rate*win_length_seconds/2) # 1/2 window shift

n_mfcc = 30 # number of desired coefficients per window
n_windows = sample_rate*3//hop_length -1 # total number of windows

# store coefficients in array
coefficients = np.zeros([len(data),n_mfcc*n_windows])

# specify MFCC transform
transform = torchaudio.transforms.MFCC(
    sample_rate = 22050,
    n_mfcc = n_mfcc,
    melkwargs={'n_fft':win_length, 'n_mels': 70, 'center': False}
)

In [8]:
# iterate through audio files, produce MFCCs for each signal
for i,filename in enumerate(data['filename']):
    waveform,_ = torchaudio.load('./archive/wavfiles/'+filename)
    coefficients[i] = transform(waveform).flatten()

In [9]:
# store features with data for later use
mfcc_dataframe = pd.concat([data[['species']],pd.DataFrame(coefficients)],axis=1)
mfcc_dataframe.to_csv('./features/mfcc.csv')

In [10]:
print(coefficients.shape)

(5422, 4470)


The dimension of the data is quite large. We will use Principal Component Analysis to reduce the size. In the future, it may be interesting to investigate whether it is better to reduce the dimension of the dataset by using a larger window size for the spectrum, or choose small window sizes and reduce the dimension afterward.

In [11]:
from sklearn.decomposition import PCA

n_components = 100 # begin with 100 components
pca = PCA(n_components = n_components)
pca.fit(coefficients)
mfcc_pca = pca.transform(coefficients)

In [12]:
mfcc_pca_dataframe = pd.concat([data[['species']],pd.DataFrame(mfcc_pca)],axis=1)
mfcc_dataframe.to_csv('./features/mfcc_pca.csv')

### 2) Chroma Features

This method extracts information from a waveform by mapping its spectrum to twelve bins representing the chromatic scale. The intuition behind this method is that it captures the harmonic characteristics of the waveform; signals of similar pitch will have similar chroma features. 



In [57]:
import librosa

chroma_features = np.zeros([len(data),12*151])

for i,filename in enumerate(data['filename']):
    waveform,_ = librosa.load('./archive/wavfiles/'+filename)
    
    chromagram = librosa.feature.chroma_stft(
    y = waveform,
    n_fft = win_length,
    hop_length = hop_length,
    sr = sample_rate
    )
    
    chroma_features[i] =  chromagram.flatten()



/Users/benjaminfrizzell/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [64]:
pca.fit(chroma_features)
chroma_features_pca = pca.transform(chroma_features)

In [68]:
chroma_feature_dataframe = pd.concat([data[['species']],pd.DataFrame(chroma_features)],axis=1)
chroma_feature_dataframe.to_csv('./features/chroma_features.csv')

chroma_feature_pca_dataframe = pd.concat([data[['species']],pd.DataFrame(chroma_features_pca)],axis=1)
chroma_feature_pca_dataframe.to_csv('./features/chroma_features_pca.csv')

### 3) Zero-Crossing Rate

We have two feature sets extracted from the frequency domain of the signals, so the final feature set we will consider will be the zero-crossing rate, which is obtained from the time domain.

In [23]:

zero_crossing_rates = np.zeros([len(data),151])

for i,filename in enumerate(data['filename']):
    waveform,_ = librosa.load('./archive/wavfiles/'+filename)
    
    zero_crossing_rates[i] = librosa.feature.zero_crossing_rate(
    y = waveform,
    frame_length = win_length,
    hop_length = hop_length,
    )


In [24]:
zero_crossing_rates.shape

(5422, 151)

The dimensionality of this feature set is reasonably small, so we will not concern ourselves with dimension reduction.

In [25]:
zcr_data = pd.concat([data[['species']],pd.DataFrame(zero_crossing_rates)],axis=1)
zcr_data.to_csv('./features/zero_crossing_rates.csv')

# Model Testing

We now have two feature sets with which to test and evaluate models. We will begin with some common classifier models:

* Random Forests
* Support Vector Machines
* Multilayer Perceptron
* K-Nearest Neighbour
* Naive Bayes

 Hyperparameters will be determined by minimizing validation error using **Bayesian Optimization**, a powerful method used for optimizing multivariate functions that have a consideral computational cost. 

Read more about the method here: https://arxiv.org/abs/1807.02811.

This is the implementation used: https://github.com/bayesian-optimization/BayesianOptimization

### Random Forest

We will start with the `RandomForestClassifier` from scikit-learn. Hyperparameters to consider with this model are:
* The number of trees in the forest: `n_estimators`
* The number of bootstrapped samples per tree: `max_samples`
* The cost-complexity pruning parameter: `ccp_alpha` (We will use post-pruning in this analysis, and not consider `max_depth`)

In [94]:
import pandas as pd

SEED = 1
mfcc_dataframe = pd.read_csv('./features/mfcc_pca.csv').iloc[:,1:]
cf_dataframe = pd.read_csv('./features/chroma_features_pca.csv').iloc[:,1:]
zcr_dataframe = pd.read_csv('./features/zero_crossing_rates.csv').iloc[:,1:]

feature_names = ['mfcc','cf','zcr']
all_feature_sets = {k:v for k,v in zip(feature_names,[mfcc_dataframe,cf_dataframe,zcr_dataframe])}

Since we have several feature sets to consider, we will split each corresponding dataframe into stratified training and testing samples, and store in a dictionary of dataframes.

In [95]:
from sklearn.model_selection import train_test_split

# split into stratified training and testing datasets
training_data = {}
testing_data = {}

for name in feature_names:
    feature_set = all_feature_sets[name]
    labels = feature_set.iloc[:,0]
    training_data[name],testing_data[name] = train_test_split(feature_set,test_size = 0.25,stratify = labels)

Before tuning hyperparameters, I would like to plot the training and testing error as a function of each hyperparameter. This may give a good idea of 'obvious' hyperparameter value regions over which to tune the models.

The error output will be stored several dictionaries, indexed by feature set type, hyperparameter, and error type.

In [115]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# dictionary to store scores
scores = {i:{j:{k:[] for k in ['train','test']} for j in ['n_estimators','ccp_alpha','max_samples']} for i in feature_names}
N_training_samples = len(training_data['mfcc'])

n_estimators_list = np.arange(75,625,25)
ccp_alpha_list = np.linspace(0,0.02,30)
max_samples_list = np.arange(500,N_training_samples+400,400)

We will now train the model over each feature set, and vary each hyperparameter independently while keeping the rest at their default values.

In [1]:
    for name in feature_names:
    
        X_train = training_data[name].iloc[:,1:]
        y_train = training_data[name].iloc[:,0]
        
        X_test = testing_data[name].iloc[:,1:]
        y_test = testing_data[name].iloc[:,0]

    # train over varying number of trees
        for n_estimators in n_estimators_list:

            model = RandomForestClassifier(
                n_estimators = n_estimators
            )
            model.fit(X_train,y_train)
            train_score = model.score(X_train,y_train)
            test_score = model.score(X_test,y_test)
            scores['n_estimators'][name]['train'].append(train_score)
            scores['n_estimators'][name]['test'].append(test_score)

    # train over varying post-pruning parameter
        for ccp_alpha in ccp_alpha_list:
        
            model = RandomForestClassifier(
                ccp_alpha = ccp_alpha
            )
            model.fit(X_train,y_train)
            train_score = model.score(X_train,y_train)
            test_score = model.score(X_test,y_test)
            scores['ccp_alpha'][name]['train'].append(train_score)
            scores['ccp_alpha'][name]['test'].append(test_score)

    # finally, train over varying number of bootstrapped samples
        for max_samples in max_samples_list:
        
            model = RandomForestClassifier(
                max_samples = max_samples
            )
            model.fit(X_train,y_train)
            train_score = model.score(X_train,y_train)
            test_score = model.score(X_test,y_test)
            scores['max_samples'][name]['train'].append(train_score)
            scores['max_samples'][name]['test'].append(test_score)
    
            

NameError: name 'feature_names' is not defined

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from bayes_opt import BayesianOptimization

# creating a wrapper for hyperparameter tuning with Bayesian Optimization
def optimize_RF(n_iter,init_points,pbounds,X,y):
    
    def cross_val_wrapper(n_estimators,max_samples,ccp_alpha):
    
        model = RandomForestClassifier(
            n_estimators = int(n_estimators),
            max_samples = int(max_samples),
            ccp_alpha = ccp_alpha
        )
    
        score = cross_val_score(model,X,y)
        return score.mean()


    optimizer = BayesianOptimization(
        f = cross_val_wrapper,
        pbounds = pbounds,
        random_state=SEED
    )


    optimizer.maximize(
        n_iter = n_iter,
        init_points = init_points
    )

    optimizer.max['params']['max_samples'] = int(optimizer.max['params']['max_samples'])
    optimizer.max['params']['n_estimators'] = int(optimizer.max['params']['n_estimators'])
    return optimizer.max


In [18]:
%%time
pbounds = {'n_estimators':(100,500),'max_samples':(400,len(data)//1.5),'ccp_alpha':(0,0.05)}
n_iter = 40
init_points = 5

mfcc_optimal_parameters = optimize_RF(n_iter,init_points,pbounds,X_mfcc,y_mfcc)

|   iter    |  target   | ccp_alpha | max_sa... | n_esti... |
-------------------------------------------------------------
| 1         | 0.3467    | 0.02085   | 2.715e+03 | 100.0     |
| 2         | 0.446     | 0.01512   | 871.7     | 136.9     |
| 3         | 0.4889    | 0.009313  | 1.511e+03 | 258.7     |
| 4         | 0.3423    | 0.02694   | 1.747e+03 | 374.1     |
| 5         | 0.4572    | 0.01022   | 3.222e+03 | 111.0     |
| 6         | 0.2316    | 0.04654   | 3.223e+03 | 109.1     |
| 7         | 0.3403    | 0.02661   | 2.931e+03 | 170.0     |
| 8         | 0.3421    | 0.02979   | 1.239e+03 | 398.7     |
| 9         | 0.4869    | 0.01138   | 900.1     | 478.9     |
| 10        | 0.337     | 0.03631   | 2.742e+03 | 177.9     |
| 11        | 0.3462    | 0.02351   | 1.761e+03 | 288.8     |
| 12        | 0.2316    | 0.04777   | 2.534e+03 | 288.3     |
| 13        | 0.3412    | 0.02444   | 3.104e+03 | 491.6     |
| 14        | 0.3534    | 0.01612   | 2.854e+03 | 442.8     |
| 15    

In [19]:
import pickle
with open('./parameters/RF_mfcc.pckl','wb') as f:
    pickle.dump(mfcc_optimal_parameters,f)